# GPT Finetuning example

In [1]:
import json
import yaml
import pandas as pd
import os
import openai

# OpenAI

## Create an OpenAI account, and generate an API_TOKEN
  - https://openai.com/api/login
  - https://beta.openai.com/account/api-keys

## Set the API KEY on openai library

In [25]:
# Uncomment the next line and replace with your API KEY
#%env OPENAI_API_KEY=YOUR_API_KEY
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_key

# Dataset

## Questions and answers related to mental health

In [2]:
with open('depression.yml', 'r') as file:
     data = yaml.safe_load(file)
        
conversations = data['conversations']
print(f"Questions number: {len(conversations)}")

Questions number: 51


## Question/Answer sample

In [3]:
conversation_index = 7
print(f"Question: {conversations[conversation_index][0]}")
print("Possible answers:")
for i, answer in enumerate(conversations[conversation_index][1:]):
    print(f"{i}. {answer}")

Question: What is Depression?
Possible answers:
0. Depression is a mood disorder that causes a persistent feeling of sadness and loss of interest. Also called major depressive disorder or clinical depression, it affects how you feel, think and behave and can lead to a variety of emotional and physical problems.


## Data preprocessing 

### Generate the jsonl file for prepare the finetuning data

In [4]:
output = []
for conversation in conversations:
    completion = ""
    for i, dialog in enumerate(conversation):
        if i == 0:
            prompt = dialog
            prompt = prompt.replace("\xa0", " ")
        else:
            completion += " " + dialog
            completion = completion.replace("\xa0", " ")
    completion = completion.strip()
    line = {'prompt': prompt, 'completion': completion}
    output.append(line)

In [5]:
output

[{'prompt': 'What Is Depression?',
  'completion': 'Depression is a common and serious medical illness that negatively affects how you feel, the way you think and how you act. Fortunately,it is also treatable. Depression causes feelings of sadness and/or a loss of interest in activities you once enjoyed. It can lead to a variety of emotional and physical problems and can decrease your ability to function at work and at home.'},
 {'prompt': 'I feel i have let my parents down',
  'completion': 'No matter what,your parents will always be proud of you and will love you. You will feel much better if you share your feelings with them.'},
 {'prompt': 'What are the types of depression?',
  'completion': 'There are many different forms of depression but most common types are. Clinical depression,persistent depressive disorder,bipolar disorder,postnatal disorder.'},
 {'prompt': 'I am good for nothing.',
  'completion': "Don't ever think you cannot achieve what you want. You are capable of reachi

### Save .jsonl file

In [6]:
with open('depression.jsonl', 'w') as outfile:
        for i in output:
            json.dump(i, outfile)
            outfile.write('\n')

# Follow the README 2, 3, 4 instructions, then follow the notebook

## Use your finetuned model 

In [21]:
model = "davinci:ft-personal-2022-12-21-20-10-56" #replace with your model name
query = 'Can you help me?'
prompt = f"""
The following is a conversation with a therapist and a user. 
The therapist is JOY, who uses compassionate listening to have helpful
and meaningful conversations with users. JOY is empathic and friendly.
JOY's objective is to make the user feel better by feeling heard. With
each response, JOY offers follow-up questions to encourage openness and
tries to continue the conversation in a natural way.\n\n
JOY-> Hello, I am your personal mental health assistant.
What's on your mind today?\nUser->{query}JOY->
"""

response = openai.Completion.create(
                #model="davinci:ft-YzSMpfIHEUwanA6JFyPLJJuQ",
                model=model,
                prompt=prompt,
                temperature=0.89,
                max_tokens=162,
                top_p=1,
                frequency_penalty=0,
                presence_penalty=0.6,
                stop=["\n"]
  )
print(f"Question: {query}")
print(f"Response: {response['choices'][0]['text']}")
#print(response)

Question: Can you help me?
Response: I am always available to help you. Tell me what you need. User-> I feel like crying. I can't sleep at night. And


In [ ]:
#https://beta.openai.com/docs/api-reference/completions/create

#Completions: Finetuning

#temperature:   What sampling temperature to use. 
                #Higher values means the model will take more risks.
                #Try 0.9 for more creative applications, and 0 (argmax sampling)
                #for ones with a well-defined answer.
            
#max_tokens: The maximum number of tokens to generate in the completion.
             #The token count of your prompt plus max_tokens cannot exceed
             #the model's context length. Most models have a context length of 2048
             #tokens (except for the newest models, which support 4096).
            
#top_p: An alternative to sampling with temperature,
        #called nucleus sampling, where the model considers
        #the results of the tokens with top_p probability mass.
        #So 0.1 means only the tokens comprising the top 10% probability mass are considered.
        #We generally recommend altering this or temperature but not both.

#frequency_penalty: Number between -2.0 and 2.0. 
        #Positive values penalize new tokens based on their
        #existing frequency in the text so far, decreasing 
        #the model's likelihood to repeat the same line verbatim.

#presence_penalty: Number between -2.0 and 2.0. 
        #Positive values penalize new tokens based on whether 
        #they appear in the text so far, increasing the model's
        #likelihood to talk about new topics.


#stop:  Up to 4 sequences where the API will stop generating further tokens. 
        #The returned text will not contain the stop sequence.
        #We generally recommend altering this or top_p but not both.